#<font color=fbb03b>翌週のサザエさんのじゃんけんを予測（LightGBMモデルを使用）

##<font color=fbb03b>事前準備

Google Driveを使用の場合は以下のセルを実行してドライブをマウントし実行ディレクトリへ移動します.

In [64]:
from google.colab import drive, output
output.clear()
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [65]:
'''データが置いてあるディレクトリのパスを入力して実行しましょう．'''
# 以下はサンプルです.ご自身の環境に合わせてフォルダへのパスは変更してください.
%cd /content/drive/My Drive/Colab Notebooks/サザエさん_じゃんけん

/content/drive/My Drive/Colab Notebooks/サザエさん_じゃんけん


###ライブラリのインポート

In [66]:
!pip install optuna-integration[lightgbm]

In [248]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from lightgbm import LGBMClassifier
from lightgbm import early_stopping

###データ準備

In [249]:
# データ読み込み
df = pd.read_csv('traindata_サザエさんじゃんけん分析.csv')

In [250]:
df

,放送回,放送日,出した手,備考
0,1,1991年11月10日,チョキ,NaN
1,2,1991年11月17日,グー,NaN
2,3,1991年11月24日,グー,NaN
3,4,1991年12月1日,パー,NaN
4,5,1991年12月8日,チョキ,NaN
...,...,...,...,...
1647,1648,2024年8月11日,グー,NaN
1648,1649,2024年8月18日,チョキ,NaN
1649,1650,2024年8月25日,グー,NaN
1650,1651,2024年9月1日,チョキ,NaN


##<font color=fbb03b>特徴量を準備

In [251]:
# 日付情報の特徴量化
df['放送日'] = pd.to_datetime(df['放送日'], format='%Y年%m月%d日')
df['年'] = df['放送日'].dt.year
df['月'] = df['放送日'].dt.month
df['日'] = df['放送日'].dt.day
df['曜日'] = df['放送日'].dt.weekday

In [252]:
df

,放送回,放送日,出した手,備考,年,月,日,曜日
0,1,1991-11-10,チョキ,NaN,1991,11,10,6
1,2,1991-11-17,グー,NaN,1991,11,17,6
2,3,1991-11-24,グー,NaN,1991,11,24,6
3,4,1991-12-01,パー,NaN,1991,12,1,6
4,5,1991-12-08,チョキ,NaN,1991,12,8,6
...,...,...,...,...,...,...,...,...
1647,1648,2024-08-11,グー,NaN,2024,8,11,6
1648,1649,2024-08-18,チョキ,NaN,2024,8,18,6
1649,1650,2024-08-25,グー,NaN,2024,8,25,6
1650,1651,2024-09-01,チョキ,NaN,2024,9,1,6


In [253]:
# ラベルエンコーディング before creating lagged features
le = LabelEncoder()
df['出した手_enc'] = le.fit_transform(df['出した手'])

In [254]:
df

,放送回,放送日,出した手,備考,年,月,日,曜日,出した手_enc
0,1,1991-11-10,チョキ,NaN,1991,11,10,6,1
1,2,1991-11-17,グー,NaN,1991,11,17,6,0
2,3,1991-11-24,グー,NaN,1991,11,24,6,0
3,4,1991-12-01,パー,NaN,1991,12,1,6,2
4,5,1991-12-08,チョキ,NaN,1991,12,8,6,1
...,...,...,...,...,...,...,...,...,...
1647,1648,2024-08-11,グー,NaN,2024,8,11,6,0
1648,1649,2024-08-18,チョキ,NaN,2024,8,18,6,1
1649,1650,2024-08-25,グー,NaN,2024,8,25,6,0
1650,1651,2024-09-01,チョキ,NaN,2024,9,1,6,1


In [ ]:
# 過去の手の特徴量化 (例: 過去全部の回)
for i in range(1, len(df)-1):
    df[f'{i}回前の手'] = df['出した手_enc'].shift(i).fillna(method='bfill').astype(int)  # Convert to integers after shifting

##<font color=fbb03b>TrainDataとTestDataに分割

In [263]:
# データ分割
X = df[['年', '月', '日', '曜日'] + [f'{i}回前の手' for i in range(len(df)-1 , 0, -1)]]
y = df['出した手_enc']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [264]:
X_train.head()

,年,月,日,曜日,1651回前の手,1650回前の手,1649回前の手,1648回前の手,1647回前の手,1646回前の手,...,10回前の手,9回前の手,8回前の手,7回前の手,6回前の手,5回前の手,4回前の手,3回前の手,2回前の手,1回前の手
306,1997,11,9,6,1,1,1,1,1,1,...,2,1,0,1,2,1,1,0,2,1
192,1995,8,13,6,1,1,1,1,1,1,...,0,1,2,1,0,2,2,0,1,1
309,1997,11,30,6,1,1,1,1,1,1,...,1,2,1,1,0,2,1,0,1,2
1360,2018,11,18,6,1,1,1,1,1,1,...,2,1,0,1,2,1,0,2,2,1
63,1993,1,24,6,1,1,1,1,1,1,...,2,1,0,2,0,1,0,2,2,0


##<font color=fbb03b>LightGBMモデル構築

In [265]:
# LightGBMモデル構築
model = LGBMClassifier(objective='multiclass', num_class=3, metric='multi_logloss', random_state=42)

###データの学習

In [266]:
# 学習
model.fit(X_train, y_train,
          eval_set=[(X_test, y_test)], # Add validation set
          eval_metric='multi_logloss', # Specify evaluation metric
          callbacks=[early_stopping(10)])

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.023644 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 4832
[LightGBM] [Info] Number of data points in the train set: 1321, number of used features: 1587
[LightGBM] [Info] Start training from score -1.113100
[LightGBM] [Info] Start training from score -1.065847
[LightGBM] [Info] Start training from score -1.120036
[LightGBM] [Info] Start training from score -7.186144
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Training until validation scores don't improve for 10 rounds
Early stopping, best iteration is:
[17]	valid_0's multi_logloss: 0.987933


LGBMClassifier(metric='multi_logloss', num_class=3, objective='multiclass',
               random_state=42)

##<font color=fbb03b>予測結果の出力

In [269]:
# 予測
y_pred = model.predict_proba(X_test)

In [274]:
# 結果出力 (例: 2024年9月8日の予測)
pred_date = pd.to_datetime('2024-09-8')

# 過去すべての回分のデータを取得
past_df = df['出した手_enc'].tail(len(df)-1).values.tolist()

pred_data = pd.DataFrame([[pred_date.year, pred_date.month, pred_date.day, pred_date.weekday()] + past_df],
                        columns=['年', '月', '日', '曜日'] + [f'{i}回前の手' for i in range(len(df)-1, 0, -1)])
pred_proba = model.predict_proba(pred_data)

print(f"2024年9月8日 グー {pred_proba[0][0]*100:.1f}%　チョキ {pred_proba[0][1]*100:.1f}%　パー {pred_proba[0][2]*100:.1f}%")

2024年9月8日 グー 32.3%　チョキ 35.8%　パー 31.8%
